<a href="https://colab.research.google.com/github/minshyee/CP1/blob/main/Experiment_model_keyBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/AIBootCamp/cp1/train_data.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
origin1 = df.copy()

In [ ]:
origin1

,text,leng,tokens,target
0,going kill sooni dont know know body mind comp...,250,"['going', 'kill', 'sooni', 'know', 'know', 'bo...",1
1,sub diary day 68 today great school stuff basi...,130,"['sub', 'diary', 'day', '68', 'today', 'great'...",0
2,supposed doive got problem years antisocial da...,710,"['supposed', 'doive', 'got', 'problem', 'years...",1
3,hey yes need help new subreddit ground subredd...,181,"['hey', 'yes', 'need', 'help', 'new', 'subredd...",0
4,non trans gender dysphoria like menstruation e...,420,"['non', 'trans', 'gender', 'dysphoria', 'like'...",0
...,...,...,...,...
185654,found combo steve minecraft forgot save vault ...,111,"['found', 'combo', 'steve', 'minecraft', 'forg...",0
185655,hate admit california texas similar loyalty yo...,264,"['hate', 'admit', 'california', 'texas', 'simi...",0
185656,people online 20000 horny mf time whats guys,44,"['people', 'online', '20000', 'horny', 'mf', '...",0
185657,fuck chemistry man gotta study thing wanna lif...,56,"['fuck', 'chemistry', 'man', 'gotta', 'study',...",0


In [ ]:
df.isnull().sum()

text      55
leng       0
tokens     0
target     0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.isnull().sum()

text      0
leng      0
tokens    0
target    0
dtype: int64

In [ ]:
df['leng'] = df.text.apply(lambda x: len(x))
df = df[df['leng']>15].reset_index(drop=True)
df

,text,leng,tokens,target
0,going kill sooni dont know know body mind comp...,250,"['going', 'kill', 'sooni', 'know', 'know', 'bo...",1
1,sub diary day 68 today great school stuff basi...,130,"['sub', 'diary', 'day', '68', 'today', 'great'...",0
2,supposed doive got problem years antisocial da...,710,"['supposed', 'doive', 'got', 'problem', 'years...",1
3,hey yes need help new subreddit ground subredd...,181,"['hey', 'yes', 'need', 'help', 'new', 'subredd...",0
4,non trans gender dysphoria like menstruation e...,420,"['non', 'trans', 'gender', 'dysphoria', 'like'...",0
...,...,...,...,...
184405,found combo steve minecraft forgot save vault ...,111,"['found', 'combo', 'steve', 'minecraft', 'forg...",0
184406,hate admit california texas similar loyalty yo...,264,"['hate', 'admit', 'california', 'texas', 'simi...",0
184407,people online 20000 horny mf time whats guys,44,"['people', 'online', '20000', 'horny', 'mf', '...",0
184408,fuck chemistry man gotta study thing wanna lif...,56,"['fuck', 'chemistry', 'man', 'gotta', 'study',...",0


In [ ]:
df['tokens'][0]

"['going', 'kill', 'sooni', 'know', 'know', 'body', 'mind', 'completely', 'sync', 'convinced', 'going', 'commit', 'brain', 'screamed', 'id', 'fucking', 'kick', 'chair', 'fucking', 'nauseated', 'cant', 'stand', 'cant', 'stand', 'breate', 'second', 'longer', 'cant', 'stand', 'trapped', 'body', 'brain', 'fucking', 'thoughts', 'need', 'talk', 'talk']"

In [ ]:
import ast # list convert
def convert_lst(x):
  x = ast.literal_eval(x)
  return len(x)

In [ ]:
convert_lst(df['tokens'][0])

['going', 'kill', 'sooni', 'know', 'know', 'body', 'mind', 'completely', 'sync', 'convinced', 'going', 'commit', 'brain', 'screamed', 'id', 'fucking', 'kick', 'chair', 'fucking', 'nauseated', 'cant', 'stand', 'cant', 'stand', 'breate', 'second', 'longer', 'cant', 'stand', 'trapped', 'body', 'brain', 'fucking', 'thoughts', 'need', 'talk', 'talk']


37

In [ ]:
df['cnt_tokens'] = df.tokens.apply(convert_lst)

,text,leng,tokens,target,cnt_tokens
5,exposing people like furry porn day 1 us3zle,44,"['exposing', 'people', 'like', 'furry', 'porn'...",0,8
14,dont understand people actively obsess big cre...,78,"['understand', 'people', 'actively', 'obsess',...",0,11
17,sleep happy brain juice strangers internet im ...,81,"['sleep', 'happy', 'brain', 'juice', 'stranger...",0,10
18,im live streaming spider man ps4 wanna drop by...,102,"['live', 'streaming', 'spider', 'man', 'ps4', ...",0,9
20,im currently 511 hope grow 7 inches stop growi...,61,"['currently', '511', 'hope', 'grow', '7', 'inc...",0,11
...,...,...,...,...,...
184390,oh fuck oh fuck mom walked masturbating time w...,77,"['oh', 'fuck', 'oh', 'fuck', 'mom', 'walked', ...",0,12
184399,ex best friendnow want reminding stress relati...,59,"['ex', 'best', 'friendnow', 'want', 'reminding...",0,8
184407,people online 20000 horny mf time whats guys,44,"['people', 'online', '20000', 'horny', 'mf', '...",0,8
184408,fuck chemistry man gotta study thing wanna lif...,56,"['fuck', 'chemistry', 'man', 'gotta', 'study',...",0,10


In [ ]:
df = df[df['cnt_tokens']>3]

In [ ]:
df.describe()

,leng,target,cnt_tokens
count,182134.000000,182134.000000,182134.000000
mean,366.047267,0.505139,52.716753
std,623.818550,0.499975,91.293974
min,16.000000,0.000000,4.000000
25%,81.000000,0.000000,12.000000
50%,175.000000,1.000000,25.000000
75%,423.000000,1.000000,61.000000
max,40014.000000,1.000000,5717.000000


In [ ]:
train_X = df.drop('target', axis=1)
train_y = df['target']

train_X.shape, train_y.shape

((182134, 4), (182134,))

In [ ]:
train_X['text'][0]

'going kill sooni dont know know body mind completely sync ive convinced im going commit brain screamed id fucking kick chair im fucking nauseated cant stand cant stand breate second longer cant stand trapped body brain fucking thoughts need talk talk'

In [ ]:
origin = pd.read_csv('/content/drive/MyDrive/AIBootCamp/cp1/Suicide_Detection.csv')

## 1. KeyBERT

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.6 MB/s 
     |████████████████████████████████| 3.5 MB 33.5 MB/s 
     |████████████████████████████████| 1.2 MB 40.7 MB/s 
     |████████████████████████████████| 67 kB 4.3 MB/s 
     |████████████████████████████████| 895 kB 40.1 MB/s 
     |████████████████████████████████| 6.8 MB 40.8 MB/s 
     |████████████████████████████████| 596 kB 58.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=b89d8402ba689146062d1ed7300248270f317ee6e39f4ed5e4bb0a6674e99756
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

### 실물구현


In [ ]:
x = [[1]]*3
x[0]

[1]

In [ ]:
doc = train_X['text']

In [ ]:
doc.isnull().sum()

0

In [ ]:
df[750:800]

,text,leng,tokens,target,cnt_tokens
757,hopei started school tuesday ive 2 panic attac...,639,"['hopei', 'started', 'school', 'tuesday', '2',...",1,91
758,stay attempted life weeks ago spent week hospi...,934,"['stay', 'attempted', 'life', 'weeks', 'ago', ...",1,141
759,want eat phone dyingi try portable charger thi...,60,"['want', 'eat', 'phone', 'dyingi', 'try', 'por...",0,10
760,im sure anymorei dont feel actually act attemp...,406,"['sure', 'anymorei', 'feel', 'actually', 'act'...",1,59
762,yall holding stupid year want chest feel free ...,167,"['yall', 'holding', 'stupid', 'year', 'want', ...",0,24
763,friends covid covid19 changed ways area partic...,493,"['friends', 'covid', 'covid19', 'changed', 'wa...",0,66
764,feel like worth anymorei wrote long post im de...,192,"['feel', 'like', 'worth', 'anymorei', 'wrote',...",1,29
765,2336 area everybodys probably sleeping quietly...,174,"['2336', 'area', 'everybodys', 'probably', 'sl...",0,24
766,reason survived time right giving second chanc...,386,"['reason', 'survived', 'time', 'right', 'givin...",1,57
767,think die live happy lifei know life pain suff...,56,"['think', 'die', 'live', 'happy', 'lifei', 'kn...",1,10


In [ ]:
%%time
n_gram_range = (3, 3)
stop_words = "english"
top_n = 5
error_idx = []

n = len(train_X)

count=[[]]*n
candidates = [[]]*n

for i in range(n):
  try:
    count[i] = CountVectorizer(ngram_range=n_gram_range).fit([doc[i]])
    # print(count)
    candidates[i] = count[i].get_feature_names_out()
    candidates[i] = candidates[i][:top_n] 

  except:
    error_idx.append(i)
    pass
print(candidates)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
len(error_idx)

2258

#### test

In [ ]:
# 3개의 단어 묶음인 단어구 추출
n_gram_range = (3, 3)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc[730]])
print(count)
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

CountVectorizer(ngram_range=(3, 3), stop_words='english')
trigram 개수 : 1
trigram 다섯개만 출력 : ['virgin alabama orphan']


## keyword 수치화

In [ ]:
# keyword 수치화
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

doc_embedding = [[]] * n
candidate_embeddings = [[]] * n
distances= [[]] * n
keywords = [[]] * n

keyword_err = []

for i in tqdm(range(n)):
  try:
    doc_embedding[i] = model.encode([doc[i]])
    candidate_embeddings[i] = model.encode(candidates[i])
    distances[i] = cosine_similarity(doc_embedding[i], candidate_embeddings[i])
    keywords[i] = [candidates[i][index] for index in distances[i].argsort()[0][-top_n:]]
  except:
    keyword_err.append(i)
    pass


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 36%|███▌      | 65074/182134 [4:39:45<7:45:14,  4.19it/s]


In [ ]:
# keyword 수치화
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([df['text'][0]])
candidate_embeddings = model.encode(candidates)

In [ ]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords

['wife good cheated',
 'threatening suicide tirelessly',
 'ex wife threatening',
 'suiciderecently left wife',
 'wife threatening suiciderecently']

## KeyBERT Library

In [ ]:
!pip install keybert

     |████████████████████████████████| 217 kB 4.9 MB/s 
     |████████████████████████████████| 51 kB 6.9 MB/s 
  Created wheel for keybert: filename=keybert-0.5.0-py3-none-any.whl size=20491 sha256=6dea3ba896dbbe1f9769ff92998b556a46b1977a3470745f6c6e5a83a37750db
  Stored in directory: /root/.cache/pip/wheels/99/1f/3f/590d2997adbb2d0e1f82e8ee05d42d6910e92c3ed283015ff8
Successfully built keybert


In [ ]:
from keybert import KeyBERT

In [ ]:
kw_model = KeyBERT()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Basic

In [ ]:
keyword = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english', top_n=5)
keyword

keyphrase_ngram_range=(1, 1)

In [ ]:
keyword = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words='english', top_n=5, highlight=True)

Ex Wife Threatening SuicideRecently I left my wife for good because she has cheated on me 
twice and lied to me so much that I have decided to refuse to go back to her. As of a few 
days ago, she began threatening suicide. I have tirelessly spent these paat few days talking 
her out of it and she keeps hesitating because she wants to believe I'll come back. I know a 
lot of people will threaten this in order to get their way, but what happens if she really 
does? What do I do and how am I supposed to handle her death on my hands? I still love my 
wife but I cannot deal with getting cheated on again and constantly feeling insecure. I'm 
worried today may be the day she does it and I hope so much it doesn't happen.

Max Sum Similarity : 데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됩니다. 여기서의 의도는 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것입니다.

In [ ]:
keyword = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=5)
keyword

[('handle death hands', 0.0041),
 ('cheated twice lied', 0.1983),
 ('wife good cheated', 0.4441),
 ('suiciderecently left', 0.0849),
 ('ex wife threatening', 0.0815)]

Maximal Marginal Relevance : 결과를 다양화하는 마지막 방법은 MMR(Maximum Limit Relegance)입니다. MMR은 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력합니다. 참고 할 수 있는 자료로 EmbedRank(https://arxiv.org/pdf/1801.04470.pdf) 라는 키워드 추출 알고리즘은 키워드/키프레이즈를 다양화하는 데 사용할 수 있는 MMR을 구현했습니다. 먼저 문서와 가장 유사한 키워드/키프레이즈를 선택합니다. 그런 다음 문서와 유사하고 이미 선택된 키워드/키프레이즈와 유사하지 않은 새로운 후보를 반복적으로 선택합니다.

In [ ]:
keyword = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words='english', use_mmr=True, diversity=0.7)
keyword

[('wife threatening suiciderecently', 0.652),
 ('cheated twice lied', 0.3034),
 ('does hope doesn', 0.1209),
 ('feeling insecure worried', 0.2322),
 ('order way happens', 0.0404)]